In [1]:
import json

data = []
with open("ocr_results.json") as thefile:
    data = json.load(thefile)

In [79]:
import re

by_store_name = {}

stores = {
  "KIWI": r"KIWI",
  "SPAR": r"SPAR",
  "REMA1000": r"REMA",
  "BUNNPRIS" : r"BUNNPRIS",
  "COOP":r"COOP",
  "MENY":r"MENY",
  "MINDE":r"MINDE",
  "LEROYMAT": r"LER[O|Ø|0]Y",
  "EUROPRIS": r"EUROPRIS",
}


# Sort by store
for str in data:
  found = False
  for k,pattern in stores.items():
    if(re.search(pattern, str)):
      by_store_name.setdefault(k,[]).append(str)
      found = True
      break


In [81]:
hit_count = [(name,len(count)) for name,count in by_store_name.items()]
hit_count.sort(key=lambda t: t[1],reverse=True)
total_count = 0

for store in hit_count:
  total_count += store[1]
  print(store)
print(total_count)
print(f"total receipts {len(data)} where {len(data) - total_count} did not match any regex")

('KIWI', 471)
('REMA1000', 161)
('SPAR', 114)
('COOP', 79)
('MENY', 77)
('BUNNPRIS', 33)
('LEROYMAT', 3)
938
total receipts 974 where 36 did not match any regex
